In [ ]:
# from jax_smi import initialise_tracking
# initialise_tracking()
import jax
from jax import config
config.update("jax_enable_x64", True)

from pathlib import Path

from matplotlib import pyplot as plt
import numpy as np

import importlib

jax.devices()

In [ ]:
import importlib
from viperleed_jax import parameter_space
from viperleed_jax.parameters import (
    vib_parameters,
    hierarchical_linear_tree,
    geo_parameters,
    occ_parameters,
)
from viperleed_jax.parameters import linear_transformer
from viperleed_jax import base_scatterers
from viperleed_jax.files.displacements import lines

from viperleed_jax.files.displacements import file

from viperleed_jax.files.displacements import regex
from viperleed_jax.files.displacements import lines
from viperleed_jax.files.displacements import file
from viperleed_jax.files.displacements import reader
from viperleed_jax.files.displacements import targeting
from viperleed_jax.parameters import linear_transformer
from viperleed_jax.parameters import vib_parameters
from viperleed_jax.parameters import geo_parameters
from viperleed_jax.parameters import meta_parameters
from viperleed_jax.parameters import occ_parameters
from viperleed_jax.parameters import hierarchical_linear_tree
from viperleed_jax import parameter_space

In [ ]:
importlib.reload(base_scatterers)
importlib.reload(regex)
importlib.reload(targeting)
importlib.reload(lines)
importlib.reload(reader)
importlib.reload(file)
importlib.reload(linear_transformer)
importlib.reload(hierarchical_linear_tree)
importlib.reload(vib_parameters)
importlib.reload(geo_parameters)
importlib.reload(meta_parameters)
importlib.reload(occ_parameters)
importlib.reload(parameter_space)

In [ ]:
path = Path(
    "/Users/alexander/GitHub/on-the-fly-deltas/tests/test_data/Pt_111_10x10_Te/converged/DISPLACEMENTS"
)
path = Path(
    "/Users/alexander/GitHub/on-the-fly-deltas/tests/test_data/displacements/Pt_111_10x10_Te/DISPLACEMENTS_z"
)

f = file.DisplacementsFile()
f.read(path)

search_block = f.blocks[0]
offsets_block = f.offsets_block()

In [ ]:
from viperleed_jax.from_state import run_viperleed_initialization

In [ ]:
origin_path = Path("../tests/test_data/Pt_111_10x10_Te/converged/")
tensor_path = Path("../large_files/Pt_111_10x10_Te/converged/Tensors_001.zip")

last_state = run_viperleed_initialization(origin_path)
slab, rpars = last_state.slab, last_state.rpars

In [ ]:
bs = base_scatterers.BaseScatterers(slab)
ps = parameter_space.ParameterSpace(bs, rpars)

In [ ]:
ps.apply_displacements(offsets_block, search_block)

In [ ]:
print(ps.info)

In [ ]:
slab.atlist

In [ ]:
%matplotlib inline

In [ ]:
from viperleed_jax.from_state import calculator_from_state
from viperleed_jax.parameter_space import ParameterSpace

In [ ]:
from viperleed.calc.files.displacements import readDISPLACEMENTS
from viperleed.calc import LOGGER as logger
from viperleed.calc.files.phaseshifts import readPHASESHIFTS
from viperleed.calc.files.iorfactor import beamlist_to_array

In [ ]:
%%time
origin_path = Path('../tests/test_data/Pt_111_10x10_Te/converged/')
tensor_path = Path('../large_files/Pt_111_10x10_Te/converged/Tensors_001.zip')

(calculator,
    slab,
    rpars,
    ref_data,
    phaseshifts,
    base_scatterers,
    disp_file_from_state,
    ps_from_state,
) = calculator_from_state(
    origin_path,
    tensor_path,
    l_max=10,
    batch=50,
    recalculate_ref_t_matrices=False,
)

In [ ]:
import pickle

with open('pt_calculator.pckl', 'wb') as f:
    pickle.dump(calculator, f)
with open('pt_slab.pckl', 'wb') as f:
    pickle.dump(slab, f)
with open('pt_rpars.pckl', 'wb') as f:
    pickle.dump(rpars, f)
with open('pt_ref_data.pckl', 'wb') as f:
    pickle.dump(ref_data, f)
with open('pt_phaseshifts.pckl', 'wb') as f:
    pickle.dump(phaseshifts, f)

In [ ]:
import importlib

In [ ]:

from viperleed_jax.parameters import geo_parameters, vib_parameters, occ_parameters, v0r_parameters, base_parameters
from viperleed_jax import dense_quantum_numbers
from viperleed_jax import base
from viperleed_jax import parameter_space as ps
from viperleed_jax import tensor_calculator as tc
from viperleed_jax import lib_delta
from viperleed_jax import batching
importlib.reload(batching)
importlib.reload(lib_delta)
importlib.reload(dense_quantum_numbers)
importlib.reload(base_parameters)
importlib.reload(vib_parameters)
importlib.reload(geo_parameters)
importlib.reload(occ_parameters)
importlib.reload(v0r_parameters)
importlib.reload(ps)
importlib.reload(base)
importlib.reload(tc)


In [ ]:
parameter_space = ps.ParameterSpace(slab)

## GEOMETRY
# freeze 3 bottom layers
parameter_space.geo_params.fix_layer(3, z_offset=0.)
#parameter_space.geo_params.fix_layer(1, z_offset=0.)
parameter_space.geo_params.fix_layer(2, z_offset=0.)

# symmetry constrained xyz movements ± 0.15 A
for param in [p for p in parameter_space.geo_params.terminal_params if p.bound is None]:
    param.set_bound(geo_parameters.GeoParamBound(-0.15, + 0.15))

## VIBRATIONS
# fix *_def
for param in [p for p in parameter_space.vib_params.terminal_params if p.site_element.site.endswith('_def')]:
    parameter_space.vib_params.fix_site_element(param.site_element, None) # None fixes to the default value

# the rest can vary ± 0.05 A
for param in [p for p in parameter_space.vib_params.terminal_params if not p.site_element.site.endswith('_def')]:
    param.set_bound(vib_parameters.VibParamBound(-0.05, + 0.05))


## CHEMISTRY
# no vacancies
parameter_space.occ_params.remove_remaining_vacancies()

# V0R
# set ± 4 eV
parameter_space.v0r_param.set_bound(v0r_parameters.V0rParamBound(-4., +4.))

In [ ]:
%%time
jax.clear_caches()
test_calc = tc.TensorLEEDCalculator(calculator.ref_data, calculator.phaseshifts, slab, rpars, batch=8)

In [ ]:
%%time
test_calc.set_parameter_space(parameter_space)

In [ ]:
%time test_calc.jit_R(np.array([0.5]*test_calc.n_free_parameters ))

In [ ]:
%time test_calc.jit_R(np.array([0.5]*test_calc.n_free_parameters ))

In [ ]:
%time test_calc.jit_R(np.array([0.5]*test_calc.n_free_parameters ))

In [ ]:
a